# Lab 1: Introduction to Neural Networks
In this tutorial we introduce some of the concepts for working with neural networks using [Pytorch](https://pytorch.org/tutorials/recipes/recipes_index.html). The entire notebook can be executed as-is, given the lack of time for this first lab session. We encourage you to explore the code yourselves to get comfortable with the concepts of deel learning in the context of biology. A few questions at the end challenge you to play around with the code and try things for yourselves.

In this session, you will create a simple neural network that classifies any given DNA sequence as protein coding or not. As a starting point, we use as examples the coding DNA sequences from humans (homo sapiens (HS)). As negatives, we use random sequences of DNA where each nucleotide is drawn from a uniform distribution over the possible nucleotides. We then train a neural network on de [codon frequencies](https://en.wikipedia.org/wiki/DNA_and_RNA_codon_tables) of these sequences. In addition to human DNA sequences, we also take a look at coding sequences from mice ([mus musculus (MM)](https://en.wikipedia.org/wiki/House_mouse)) and yeast ([saccharomyces cerevisiae (SC)](https://en.wikipedia.org/wiki/Saccharomyces_cerevisiae)). 


In [ ]:

# import pytorch
import torch
import torch.nn as nn
from torch import Tensor
from torch import optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from sklearn.model_selection import KFold

# import basic functionality
import random
import numpy as np
import pandas as pd

# libraries for plotting
import seaborn as sns
import matplotlib.pyplot as plt

import _pickle as pickle

import Bio
from Bio import SeqIO

# Step 1: Pre-processing the data
Here we download and pre-process the dataset. We only consider DNA sequences that are protein coding, contain a integer number of codons, and have a start and stop codon. Finally, we remove duplicates and randomly mix the sequences. 

In [ ]:

# download and unpack DNA coding sequences for human, mouse and yeast
############################

!mkdir -p ~/all_seqs
%cd ~/

!wget -P ~/all_seqs/ https://ftp.ensembl.org/pub/current_fasta/homo_sapiens/cds/Homo_sapiens.GRCh38.cds.all.fa.gz
!gzip -df "all_seqs/Homo_sapiens.GRCh38.cds.all.fa.gz"

!wget -P ~/all_seqs/ https://ftp.ensembl.org/pub/current_fasta/saccharomyces_cerevisiae/cds/Saccharomyces_cerevisiae.R64-1-1.cds.all.fa.gz
!gzip -df "all_seqs/Saccharomyces_cerevisiae.R64-1-1.cds.all.fa.gz"

!wget -P ~/all_seqs/ https://ftp.ensembl.org/pub/current_fasta/mus_musculus/cds/Mus_musculus.GRCm39.cds.all.fa.gz
!gzip -df "all_seqs/Mus_musculus.GRCm39.cds.all.fa.gz"


In [ ]:

############################
### some unnecessarily complex code to process the FASTA files for coding sequences
############################

# function that loads and processes a FASTA file containing coding sequences
def load_species_cds(file_name, max_nr_samples):
    seqs = []
    for record in SeqIO.parse(file_name, "fasta"):
        # ensure that sequences are protein coding
        if 'gene_biotype:protein_coding' in record.description:
            if 'transcript_biotype:protein_coding' in record.description:
                if ' cds ' in record.description:
                    if len(record.seq) % 3 == 0:
                        # translate sequence and check for start and stop codons
                        code_translation = str(record.seq.translate())
                        if (code_translation[0]=='M') & (code_translation[-1]=='*'):
                            seqs.append(str(record.seq))

    # avoid sequences with undetermined/uncertain nucleotides
    # restrict to sequences with at least 100 aa for codon frequency estimation
    seqs = [seqs[i] for i in range(len(seqs)) if (len(seqs[i])>=300)]#('N' not in train_cds_filtered[i]) and (len(train_cds_filtered[i])>=300)]
    
    # remove duplicates and randomly mix the list of sequences
    seqs = list(set(seqs))
    random.shuffle(seqs)
    
    return list(seqs)[0:max_nr_samples]


In [ ]:

# there are many sequences, given the time constraints, we limit the number of sequences to speed up the processing
max_nr_samples = 20000

# load coding sequences for different species
print('loading human proteins')
seq_data_human = load_species_cds("all_seqs/Homo_sapiens.GRCh38.cds.all.fa", max_nr_samples)

print('loading yeast proteins')
seq_data_yeast = load_species_cds("all_seqs/Saccharomyces_cerevisiae.R64-1-1.cds.all.fa", max_nr_samples)

print('loading mouse proteins')
seq_data_mouse = load_species_cds("all_seqs/Mus_musculus.GRCm39.cds.all.fa", max_nr_samples)

# take a look at some sequences
[seq_data_human[i][0:20]+'...'+seq_data_human[i][-20:] for i in range(5)]


# Step 2: Encoding sequences as codon frequencies
The steps above give us a set of unique coding sequences for humans, mice and yeast. To train a neural network on de coding frequencies of these sequences, we encode the sequences by converting each sequence to an array of frequencies for each possible codon. Each codon gets assigned a index in the array. We first create a 'language' that knows all possible words (codons) for a given codon length and input sequence. This language then allows us to convert between codon (e.g., 'ATG') and indices in the array (e.g., 'ATG' -> 0) to keep track of the codon frequencies. Here, we use Tensors - the datatype used for pytorch data - to store the coding frequencies.

Biologically, [DNA codons](https://en.wikipedia.org/wiki/DNA_and_RNA_codon_tables) consist of three nucleotides, encoding amino acids. However, since we are training a neural network to classify a sequence to be protein coding or not, we can choose any number of nucleotides to represent a 'codon'. For example, we can choose a "codon length" (codon_length) of a single nucleotide (which would result in us training the model on the [frequencies of nucleotides in DNA](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC403801/)), or a codon length of two nucleotides (no biological meaning as this does not represent a biological unit - we do not expect a model to learn any biology at all), or a codon length of 6 nucleotides (representing pairs of amino acids - would this yield a model that "learns" any biology?). You can play around with the codon_length yourself, but we start with a codon length of 3 nucleotides - represeting one amino acid. 

In [ ]:

# class to store a language
class Language:
    # initialize the language, as standard we have start of sentence (SOS), end of sentence (EOS) and a padding to equalize sentence lengths (PAD)
    def __init__(self, name, codon_len):
        self.name = name
        self.word2index = {}
        self.index2word = {}
        self.n_words = 0
        self.codon_length = codon_len


    # function to split sentence in blocks of a given codon_length
    def splitSentence(self, sentence):
        return [sentence[i:i+self.codon_length] for i in range(0,len(sentence),self.codon_length) if len(sentence[i:i+self.codon_length])==self.codon_length]


    # function to add sentence to language (add all new words in the sentence to our language)
    def learnWords(self, sentence):
        sentence_split = self.splitSentence(sentence)
        for word in sentence_split:
            if word not in self.word2index:
                self.word2index[word] = self.n_words
                self.index2word[self.n_words] = word
                self.n_words += 1


    # function to count the word frequencies in a sentence
    def encode(self, sentence):
        sentence_split = self.splitSentence(sentence)
    
        ############################
        ### create a tensor having the number of available words as length, fill with with zeros
        codon_freqs = torch.zeros(self.n_words)
        ############################

        # count frequencies of every word in the sentence
        word_counts = np.unique(sentence_split, return_counts = True)
        for i in range(len(word_counts[0])):
            codon_freqs[self.word2index[word_counts[0][i]]] = word_counts[1][i]
        codon_freqs /= len(sentence_split)

        return codon_freqs


    # return a sample of frequencies for all words in a language
    # here we're matching the codon frequencies to the sequence length of the provided sequence
    def sample_sentence(self, sentence):
        # create a tensor having the number of available words as length, fill with with zeros
        codon_freqs = torch.zeros(self.n_words)

        # sample nr of codons in sequence based on actual data
        # generate random sequence of codons given the current sequence length
        nr_codons = round(len(sentence)/self.codon_length)
        sampled_codons = list(np.random.randint(low=0, high=self.n_words, size = nr_codons, dtype=int))

        # count frequencies of every codon
        word_counts = np.unique(sampled_codons, return_counts = True)
        for i in range(len(word_counts[0])):
            codon_freqs[word_counts[0][i]] = word_counts[1][i]
        codon_freqs /= nr_codons

        return codon_freqs

    
    # here we define a function for encoding an entire dataset sequences
    def encode_dataset(self, dataset):
        # encode positives
        encoded_positives = [{'sample':self.encode(sentence),'label':torch.Tensor([1])} for sentence in dataset]

        # sample negatives following the sequence length distribution of positives
        encoded_negatives = [{'sample':self.sample_sentence(sentence),'label':torch.Tensor([0])} for sentence in dataset]

        # merge datasets and randomly mix positives and negatives
        dataset_encoded = encoded_positives + encoded_negatives
        random.shuffle(dataset_encoded)
        
        return dataset_encoded
        

    # here we define a function for encoding an entire dataset sequences
    def encode_positives(self, dataset):
        # encode positives
        encoded_positives = [{'sequence':sentence,'frequencies':self.encode(sentence),'label':torch.Tensor([1])} for sentence in dataset]
        return encoded_positives


    # here we define a function for encoding an entire dataset sequences
    def encode_negatives(self, dataset):
        # sample negatives following the sequence length distribution of positives
        encoded_negatives = [{'sequence':sentence,'frequencies':self.sample_sentence(sentence),'label':torch.Tensor([0])} for sentence in dataset]
        return encoded_negatives



In [ ]:

codon_length = 3

# create a language for human DNA sequences
dna_lang = Language(name="dna_human", codon_len=codon_length)

# memorize the dna language by parsing all sequences
for cur_seq in seq_data_human:
    dna_lang.learnWords(cur_seq)

# encode data
seq_data_human_encoded_pos = dna_lang.encode_positives(seq_data_human)
seq_data_human_encoded_neg = dna_lang.encode_negatives(seq_data_human)
seq_data_mouse_encoded = dna_lang.encode_positives(seq_data_mouse)
seq_data_yeast_encoded = dna_lang.encode_positives(seq_data_yeast)


In [ ]:

# store output
with open('lab1_seq_data_human_encoded_pos.obj', "wb") as file_handler:
        pickle.dump(seq_data_human_encoded_pos, file_handler)

# store output
with open('lab1_seq_data_human_encoded_neg.obj', "wb") as file_handler:
        pickle.dump(seq_data_human_encoded_neg, file_handler)

# store output
with open('seq_data_mouse_encoded.obj', "wb") as file_handler:
        pickle.dump(seq_data_mouse_encoded, file_handler)

# store output
with open('seq_data_yeast_encoded.obj', "wb") as file_handler:
        pickle.dump(seq_data_yeast_encoded, file_handler)


In [ ]:

# store output
with open('sequence_encoding.obj', "wb") as file_handler:
        pickle.dump(dna_lang, file_handler)
